In [1]:
from google import genai
from google.genai import types
import base64
from openai import OpenAI
import requests
import json
from src.apicall import get_places_details

In [8]:
place_details=get_places_details(coordinate=(52.523249, 13.392378),radius=1000,place_type="cafe")

json_input=place_details[[
        "current_opening_hours_open_now",
        "reservable",
        "curbside_pickup",
        "price_level",
        "rating",
        "serves_breakfast",
        "serves_vegetarian_food",
        "editorial_summary_overview",
        "serves_lunch",
        "utc_offset",
        "takeout",
        "delivery",
        "dine_in",
        "wheelchair_accessible_entrance",
        "name",
        "serves_beer",
        "serves_brunch",
        "user_ratings_total",
        "serves_wine",
        'cashpayment','cardpayment',
'dog_friendly', 'laptop_friendly',
        "serves_dinner",
        "reviews",
        "business_status"]]
json_input=json_input[json_input["name"]=="YOSOY TAPAS BERLIN"]

llm_input=json_input.to_json()


In [ ]:
place_details=get_places_details(coordinate=(52.523249, 13.392378),radius=1000,place_type="cafe")

json_input=place_details[[
        "current_opening_hours_open_now",
        "reservable",
        "curbside_pickup",
        "price_level",
        "rating",
        "serves_breakfast",
        "serves_vegetarian_food",
        "editorial_summary_overview",
        "serves_lunch",
        "utc_offset",
        "takeout",
        "delivery",
        "dine_in",
        "wheelchair_accessible_entrance",
        "name",
        "serves_beer",
        "serves_brunch",
        "user_ratings_total",
        "serves_wine",
        'cashpayment','cardpayment',
'dog_friendly', 'laptop_friendly',
        "serves_dinner",
        "reviews",
        "business_status"]]
  
llm_input=json_input.to_json()


In [9]:



def convert_place_info(place_json):
  """
  Input `place_json`: a JSON file with place information + reviews
  Output: text
  """
  
  with open(place_json, 'r', encoding='utf-8') as file:
    data = json.load(file)  # Load JSON data into a Python object
  
  text_output = json.dumps(data, indent=4)
  
  return text_output


def compare_requirements_with_place_info(requirements=None, place_details=None):
  client = genai.Client(
    vertexai=True,
    project="bliss-hack25fra-9533",
    location="us-central1",
  )

  text1 = types.Part.from_text(text=
                               """You are an expert travel advisor, and you love to make personalized recommendations for users.
                              
                               A user gives you his requirements for the perfect cafe {requirements}.
                               Your task is to review the user input json based on the requirements and find the best match. Focus on the requirements the relevant columns in the user input and the reviews.
                               Your task is to give the user a rating between "poor fit","okay","good","very good","perfekt fit" starts of how well does your favorite cafe match his requirements.
                               Ensure to check for every requirement individually.
                               example: If the requirement is asking for cash payment, you should look in the respective column.

                               Output Format:
                               - Valid JSON with keys 'name','rating','reasoning'
                               - ONLY include the mentioned requirements in your reasoning. Example: If "Serves beer" is not in the requirement, you should not use it in the reasoning or your rating.
                               - The json should be ordered with the highest rating on top. Be as precise as possible with the reasoning for your rating. The rating must inclue a justification about each requirement and in how far the restaurant fulfills the requiremet or not.
                               - A good example of a reasoning for the requirements "cheap", "laptop friendly","vegetarian", "pay by card", " cappuchino" would be: The restaurant appears to have affordable prices indicated by the column 'price_level'. Moreover, it serves vegetarian food as indicated by the column "serves_vegetarian_food". The restaurant further allows card payment, indicated by the column "cardpayment". Moreover, the restaurant serves cappuchino as indicated by  the column "serves_coffee". hence, the restaurant received a rating 'very good'"
                               - bring some variation and do not just write and, and, and like     "reasoning": "The restaurant serves sushi, which is a Japanese cuisine. The restaurant is also open now. The restaurant is also wheelchair accessible. The restaurant also serves vegetarian food. The restaurant is also dog friendly. The restaurant is also laptop friendly. The restaurant also serves beer and wine. The restaurant also has dine in and takeout options. Hence, the restaurant received a rating of 5 stars." Make the reasoning personal like you would recommend it a friend.
                              -  example of a poor output { "name": "YOSOY TAPAS BERLIN",
    "rating": very good,
    "reasoning": "The restaurant is known for its tapas, which aligns with the user's preference for a cafe. It is also dog-friendly, which is a plus. The restaurant also has dine-in options. The reviews mention that the food is delicious and the staff is lovely. The restaurant does not have a laptop friendly environment."
  }, here several requirements were not included in the reasoning and some requiremnets not fillfilled, therefore it cannot be high up in the ranking. Moreover, when asked for a coffee, a cafe place is a better fit. "The restaurant does not have a laptop friendly environment." clearly indicates that it is not a good match.
                                
                                Include in the reasoning every requirement. For instance it the requirements are 'serves beer', 'laptop friendly' and 'serves coffee', your reasoning should be 'it serves beer, is not laptop friendly and a a cafe which serves delicious coffee'.
                                if i ask for serves beer you should return the value for the column serves_beer in the reasoning (if it is missing just say it is not mentioned) this answer is not good as it does not mention serves beer in the reasoning   {
    "name": "The Barn Caf\u00e9",
    "rating": good,
    "reasoning": "The Barn Caf\u00e9 is a great choice for your cafe needs. It is a cafe, which means it serves coffee, and it is also dog-friendly, which is a plus. The cafe is also open now, which is convenient. The reviews mention a chilled atmosphere, and the staff is sweet and helpful. The cafe also serves vegetarian food, which is a plus. The cafe is not laptop friendly, but it is a great place to enjoy a coffee and relax. Hence, the restaurant received a rating of 4.8 stars."
  },
  An example where you can make it sound more naturally is from "The cafe also serves vegetarian food, which is a plus. laptop friendly: The cafe is laptop friendly, which is great if you want to work or study. serves beer: not mentioned. " to  " the coffee is laptop friendly, however it oes not mention if it serves beer or not.
                                """)

  
  text2 = types.Part.from_text(text=place_details)

  # place_info = convert_place_info(place_info)

  # text3 = types.Part.from_text(text=place_info)



  model = "gemini-2.0-flash-lite-001"
  contents = [
    types.Content(
      role="user",
      parts=[
        text1,
        text2,
      ]
    )
  ]
  generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_k = 1,
    candidate_count = 1,
    max_output_tokens = 8192,
  )

  response = client.models.generate_content(
    model = model,
    contents = contents,
    config = generate_content_config,
  )

  print(response.text)

  return response.text
  #return response.text


compare_requirements_with_place_info(requirements=["dog-friendly","serves cappucchino","laptop friendly","vegetarian", "pay by card","cheap"], place_details=llm_input
                                     )


c:\Users\yannick.deepen\AppData\Local\miniconda3\envs\gcp-hackathon\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


```json
[
  {
    "name": "YOSOY TAPAS BERLIN",
    "rating": "very good",
    "reasoning": "YOSOY TAPAS BERLIN is a great choice for a relaxed evening. It is currently open, which is convenient. The restaurant serves beer and wine, which is a plus. It also accepts both card and cash payments, offering flexibility. The reviews highlight the delicious food and lovely staff, creating a welcoming atmosphere. However, it is not laptop friendly, so it might not be the best place if you are looking to work or study. The price level is 2, indicating affordable prices. Hence, the restaurant received a rating of 'very good'."
  }
]
```


'```json\n[\n  {\n    "name": "YOSOY TAPAS BERLIN",\n    "rating": "very good",\n    "reasoning": "YOSOY TAPAS BERLIN is a great choice for a relaxed evening. It is currently open, which is convenient. The restaurant serves beer and wine, which is a plus. It also accepts both card and cash payments, offering flexibility. The reviews highlight the delicious food and lovely staff, creating a welcoming atmosphere. However, it is not laptop friendly, so it might not be the best place if you are looking to work or study. The price level is 2, indicating affordable prices. Hence, the restaurant received a rating of \'very good\'."\n  }\n]\n```'